In [1]:
# Create mock recommendation data for exploration

In [2]:
import numpy as np
import random
import time
import csv

In [3]:
# Create co-occurrence matrix

def create_cocmatrix(subset_matrix_cust_order):
    rows, cols = subset_matrix_cust_order.shape
    m = np.zeros((cols,cols))
    for i in range(cols):
        t = np.sum(subset_matrix_cust_order[subset_matrix_cust_order[:,i] > 0],axis=0)
        t[i] = 0
        m[i,:] = t
        if i % (cols/5) == 0:
            print "created rows in cooccurrence matrix at row",i
    return m

In [8]:
# initialize all data for collaborative

# customer to order lookup
#matrix_co = create_mock_cust_orders(NUM_OF_CUSTOMERS,NUM_OF_PRODUCTS,0.01)
matrix_co = np.load('../../data/extracts/cust_item_matrix.npy')

print "created customer to order lookup"

# load in customer ids
list_cust_ids = list(np.load('../../data/extracts/custids.npy'))

# load in asin names
list_asin_names = list(np.load('../../data/extracts/asin.npy'))

# load in max customer purchase date
list_cust_pm = list(np.load('../../data/extracts/orderdate_indexed.npy'))


created customer to order lookup


In [9]:
# generate recommendations
# purchase_vec = customer purchases (vector)
# cocm = co occurrence matrix [items x items]
# num_rec = number of recommendations
# custid = customer id you are recommending for
def gen_recom(purchase_list,cocm,num_rec):
    #print "generating",num_rec,"recommendations for customer in category",list_catid
    
    # find purchase asins/indexes
    #print "purchase indexes/values:"
    #purchase_prodid = np.where(purchase_vec > 0)[0] 
    #print purchase_prodid
    
    # add up co occurrence rows 
    #print "summed co-occurence vector:"
    rowsum = np.zeros(cocm.shape[0])
    for p in purchase_list:
        rowsum += cocm[p,:]
    #rowsum = np.sum(cocm[purchase_list,:],axis=0)
    #print rowsum
    
    # remove not in category
    #print "remove not in category"
    # do nothing if category 0 which is general
    #if len(list_catid) != 0:
    #    catarray = np.zeros(purchase_vec.shape[0])
    #    catarray[get_catarray(mpxc,list_catid)] = 1
    #    rowsum = np.multiply(rowsum,catarray)
    #else:
    #    print "no category removed general"
    #print rowsum
    
    # remove already purchased items
    #print "removed already purchased items:"
    #rowsum = np.multiply(rowsum,1-np.where(purchase_vec > 0, 1,0))
    rowsum[purchase_list] = 0
    #print rowsum
    
    # find largest indexes
    #toprec = np.argsort(rowsum)[-1 * num_rec:][::-1]
    indices = np.nonzero(rowsum)[0]
    toprec = indices[np.argsort(rowsum[indices])][-1 * num_rec:][::-1]
    #print "top",num_rec,"recommendation indexes:"
    #print toprec
    #print "top",num_rec,"recommendation indexes scores:"
    #print rowsum[toprec]
    
    return toprec


In [24]:
# put matrixes in to get cross validation recall
# generate recommendations
# mco = matrix of customer to order [customer x items]
# num_rec = number of recommendations
# num_mock = of mock recommendations
def mock_log_recommendation_generator(mco,num_rec,num_mock,l_c,l_a,l_d):
    start = time.time()
    list_rec = []
    coo_matrix = create_cocmatrix(mco)
    
    for p in range(num_mock):
        list_original_purchases = np.where(mco[p,:] > 0)[0]
        #print list_original_purchases
        toprec = gen_recom(list_original_purchases,coo_matrix,num_rec)
        
        for i,tr in enumerate(toprec):
            list_rec.append((l_c[p],l_a[tr],num_rec-i,l_d[p].strftime('%Y-%m-%d')))
    end = time.time()
    print(end - start)
    return list_rec
        
    
    

In [25]:
# generate log recomendations
ll = mock_log_recommendation_generator(matrix_co,10,matrix_co.shape[0],list_cust_ids,list_asin_names,list_cust_pm)



created rows in cooccurrence matrix at row 0
created rows in cooccurrence matrix at row 798
created rows in cooccurrence matrix at row 1596
created rows in cooccurrence matrix at row 2394
created rows in cooccurrence matrix at row 3192
72.654389143


In [26]:
ll[:100]
    

[(1, '0004115724', 10, '2012-03-09'),
 (1, '0007178735', 9, '2012-03-09'),
 (1, '0007143540', 8, '2012-03-09'),
 (1, '0006749526', 7, '2012-03-09'),
 (1, '0003708861', 6, '2012-03-09'),
 (1, '0003905276', 5, '2012-03-09'),
 (1, '0004490606', 4, '2012-03-09'),
 (1, '0004490266', 3, '2012-03-09'),
 (1, '0003271323', 2, '2012-03-09'),
 (1, '0004125193', 1, '2012-03-09'),
 (2, '0003261301', 10, '2011-02-18'),
 (2, '000611718X', 9, '2011-02-18'),
 (2, '0007178735', 8, '2011-02-18'),
 (2, '0004588266', 7, '2011-02-18'),
 (2, '0000041696', 6, '2011-02-18'),
 (2, '0004490266', 5, '2011-02-18'),
 (2, '0003709493', 4, '2011-02-18'),
 (2, '0004490940', 3, '2011-02-18'),
 (2, '0003801055', 2, '2011-02-18'),
 (2, '0004490401', 1, '2011-02-18'),
 (3, '0007178735', 10, '2014-04-30'),
 (3, '000722849X', 9, '2014-04-30'),
 (3, '0005235863', 8, '2014-04-30'),
 (3, '0005154111', 7, '2014-04-30'),
 (3, '0007178700', 6, '2014-04-30'),
 (3, '0007182147', 5, '2014-04-30'),
 (3, '0006751032', 4, '2014-04-30')

In [27]:
with open('../../data/derived/log_rec.csv','wb') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['customerid','productid','rank','timestamp'])
    for row in ll:
        csv_out.writerow(row)

In [13]:
list_asin_names

['0007176953',
 '0002160587',
 '0006380905',
 '0007127251',
 '0002118572',
 '0007148224',
 '0007229356',
 '0003271323',
 '0002218615',
 '0002551500',
 '0007183453',
 '0007173172',
 '0006472648',
 '0006184936',
 '0007136447',
 '0007226519',
 '0007334877',
 '0006551742',
 '0007181787',
 '0002240017',
 '0007211805',
 '0007193084',
 '0007254008',
 '0007178522',
 '0002112388',
 '0007277016',
 '0007175337',
 '0007149395',
 '0003273091',
 '0007382855',
 '0002219360',
 '0004351592',
 '0006531172',
 '0003830683',
 '0002430185',
 '0006753981',
 '0004125509',
 '0006394272',
 '0007103328',
 '0006754538',
 '0007165048',
 '0001057324',
 '0002557592',
 '0006279554',
 '0006395694',
 '0007183038',
 '0007284624',
 '0004701208',
 '0007301596',
 '0002163292',
 '0006243592',
 '000664659X',
 '0007280807',
 '0007214723',
 '0002740303',
 '0007305850',
 '0002554151',
 '0002728494',
 '0007112793',
 '0002176475',
 '0007201109',
 '0004133935',
 '0006641288',
 '0004126483',
 '000714668X',
 '0002188899',
 '00074215